# **AI Valorant Rank Prediction**

## **Imports & Load**

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

data = pd.read_csv('agents.csv')

## **Data Visualization**

In [2]:
data.head()

,agent,pick_rate,rounds,rating,ACS,K/D,ADR,KPR,DPR,APR,FBPR,FDPR,HS%,FBSR%
0,Jett,78.14%,462467,1.08,232.8,1.10,148.7,0.82,0.74,0.16,0.19,0.16,25%,54%
1,Sova,76.00%,450157,0.97,194.6,0.98,133.5,0.66,0.68,0.32,0.06,0.06,25%,49%
2,Astra,48.14%,284853,0.98,186.6,0.99,123.8,0.68,0.68,0.32,0.07,0.07,26%,48%
3,Viper,42.67%,253988,0.95,193.1,0.97,128.4,0.67,0.69,0.28,0.08,0.09,25%,46%
4,Killjoy,34.27%,202132,0.91,197.4,1.00,134.3,0.69,0.69,0.18,0.08,0.09,22%,48%


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   agent      18 non-null     object 
 1   pick_rate  18 non-null     object 
 2   rounds     18 non-null     int64  
 3   rating     18 non-null     float64
 4   ACS        18 non-null     float64
 5   K/D        18 non-null     float64
 6   ADR        18 non-null     float64
 7   KPR        18 non-null     float64
 8   DPR        18 non-null     float64
 9   APR        18 non-null     float64
 10  FBPR       18 non-null     float64
 11  FDPR       18 non-null     float64
 12  HS%        18 non-null     object 
 13  FBSR%      18 non-null     object 
dtypes: float64(9), int64(1), object(4)
memory usage: 2.1+ KB


## **Data Cleaning**

In [4]:
data.drop(['agent', 'pick_rate'], axis=1, inplace=True)
data['HS%'] = data['HS%'].str.replace('%', '').astype(float)
data['FBSR%'] = data['FBSR%'].str.replace('%', '').astype(float)

In [5]:
print(data.isnull().sum())

rounds    0
rating    0
ACS       0
K/D       0
ADR       0
KPR       0
DPR       0
APR       0
FBPR      0
FDPR      0
HS%       0
FBSR%     0
dtype: int64


## **Processed Data**

In [6]:
data.info()

data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   rounds  18 non-null     int64  
 1   rating  18 non-null     float64
 2   ACS     18 non-null     float64
 3   K/D     18 non-null     float64
 4   ADR     18 non-null     float64
 5   KPR     18 non-null     float64
 6   DPR     18 non-null     float64
 7   APR     18 non-null     float64
 8   FBPR    18 non-null     float64
 9   FDPR    18 non-null     float64
 10  HS%     18 non-null     float64
 11  FBSR%   18 non-null     float64
dtypes: float64(11), int64(1)
memory usage: 1.8 KB


,rounds,rating,ACS,K/D,ADR,KPR,DPR,APR,FBPR,FDPR,HS%,FBSR%
count,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000
mean,164091.888889,0.981111,201.588889,0.984444,133.077778,0.704444,0.715556,0.257778,0.100556,0.104444,24.444444,48.500000
std,133889.186290,0.058801,19.217483,0.062140,12.056902,0.062235,0.027273,0.088088,0.038877,0.031478,2.092626,2.915476
min,1978.000000,0.900000,175.000000,0.900000,117.100000,0.630000,0.680000,0.150000,0.060000,0.060000,18.000000,45.000000
25%,82658.250000,0.932500,186.850000,0.942500,124.050000,0.660000,0.700000,0.180000,0.070000,0.082500,24.250000,46.250000
50%,147818.000000,0.975000,194.050000,0.975000,130.000000,0.675000,0.705000,0.255000,0.080000,0.095000,25.000000,48.000000
75%,203398.750000,1.020000,217.825000,1.037500,142.375000,0.765000,0.740000,0.320000,0.130000,0.120000,25.750000,49.750000
max,462467.000000,1.080000,236.100000,1.100000,156.600000,0.820000,0.760000,0.430000,0.190000,0.170000,27.000000,55.000000


## **Model Training using Multiple Linear Regression**

In [7]:
X = data.drop(['rating'], axis=1)
y = data['rating']

print(X.shape)
print(y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

prediction = model.predict(X_test)

(18, 11)
(18,)


## **Ranking Logic**

In [8]:
ranks = {
    "Iron": 0,
    "Bronze": 0.5,
    "Silver": 1,
    "Gold": 1.5,
    "Platinum": 2,
    "Diamond": 2.5,
    "Immortal": 3,
    "Radiant": 3.5
}

average = np.mean(prediction)
                
def fuzzy(value, ranks):
    for rank, holder in ranks.items():
        if value < holder:
            return rank
        elif value >= holder and rank == "Radiant":
            return rank
    return "Error"
    
predicted_rank = fuzzy(average, ranks)

## **Prediction**

In [9]:
print("Predicted Rank:", predicted_rank)

Predicted Rank: Gold


## **Performance Measures**

In [10]:
print(mean_squared_error(y_test, prediction))

0.0004097323875198423
